In [198]:
max_words = 200
max_len = 40
num_classes = 6

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [199]:
import pandas as pd

df = pd.read_excel(r"C:\Users\vmakh\OneDrive\Desktop\Python-20220419T075236Z-001\Python\Neuron Networks\NLP\lection6_materials\lection6_materials\task.xls")


In [200]:
df.shape

(20659, 3)

In [201]:
df = df.rename(columns={'Content': 'text'})
df = df.rename(columns={'Rating': 'rating'})
df.head()

,rating,text,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [202]:
df['rating'].value_counts()

rating
5    14586
1     2276
4     2138
3      911
2      748
Name: count, dtype: int64

In [203]:
from sklearn.model_selection import train_test_split
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [204]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['text'] = df['text'].apply(preprocess_text)


In [205]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['rating'], test_size=0.2, random_state=42)

In [206]:
y_train.shape, y_test.shape

((16527,), (4132,))

In [207]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [208]:
corpus[:100]

'it just works целое удобноной приложениеиз минус хотеть большой доступ персональный данные телефонеп'

In [209]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vmakh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [210]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [211]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [212]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [213]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [214]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [215]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
X_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [216]:
X_train.shape, X_test.shape

((16527, 40), (4132, 40))

In [217]:
X_train[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0, 56, 44, 45, 20,  1])

In [218]:
y_train.shape, y_test.shape

((16527,), (4132,))

In [219]:
num_classes = 6
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [220]:
y_train.shape, y_test.shape

((16527, 6), (4132, 6))

In [221]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D 
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import categorical_crossentropy

#from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [222]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [223]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [224]:
tensorboard = TensorBoard(log_dir=r'C:\Users\vmakh\OneDrive\Desktop\Python-20220419T075236Z-001\Python\Neuron Networks\NLP\lection7\lection7\logs', write_graph=True, write_images=True)

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
30/30 [==============================] - 14s 404ms/step - loss: 1.2556 - accuracy: 0.6946 - val_loss: 1.0051 - val_accuracy: 0.7072
Epoch 2/20
30/30 [==============================] - 13s 425ms/step - loss: 0.8986 - accuracy: 0.7049 - val_loss: 0.8202 - val_accuracy: 0.7072
Epoch 3/20
30/30 [==============================] - 12s 411ms/step - loss: 0.7904 - accuracy: 0.7049 - val_loss: 0.7771 - val_accuracy: 0.7072
Epoch 4/20
30/30 [==============================] - 34s 1s/step - loss: 0.7567 - accuracy: 0.7048 - val_loss: 0.7634 - val_accuracy: 0.7072
Epoch 5/20
30/30 [==============================] - 23s 751ms/step - loss: 0.7430 - accuracy: 0.7062 - val_loss: 0.7530 - val_accuracy: 0.7072
Epoch 6/20
30/30 [==============================] - 26s 868ms/step - loss: 0.7322 - accuracy: 0.7075 - val_loss: 0.7496 - val_accuracy: 0.7157
Epoch 7/20
30/30 [==============================] - 33s 1s/step - loss: 0.7244 - accuracy: 0.7121 - val_loss: 0.7431 - val_accuracy: 0.7151
Epoch

In [225]:
X_train.shape, y_train.shape

((16527, 40), (16527, 6))

In [226]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

9/9 [==============================] - 2s 230ms/step - loss: 0.7028 - accuracy: 0.7611


Test score: 0.702793538570404
Test accuracy: 0.7611325979232788


In [229]:
X_test.shape, y_test.shape

((4132, 40), (4132, 6))

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score 

In [79]:
vect = TfidfVectorizer(ngram_range=(1, 2), analyzer='word', lowercase=False)

In [80]:
train_ft = vect.fit_transform(df['text'])

In [81]:
lgr = LogisticRegression()

In [82]:
lgr.fit(train_ft, df['rating'].to_numpy())

C:\Users\vmakh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [232]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import gensim.downloader as api

maxlen=40

# Step 2: Load the pre-trained Word2Vec model
model = api.load("word2vec-ruscorpora-300")

# Step 3: Initialize the embedding matrix with pre-trained vectors
embedding_dim = 300  # Dimension of the word embeddings
vocab_size = len(vocabulary) + 1  # Vocabulary size

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in vocabulary.items():
    if word in model.key_to_index:
        embedding_matrix[idx] = model[word]

# Step 4: Define the model architecture
model = tf.keras.Sequential()
model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False))
model.add(layers.LSTM(128, input_shape=(maxlen, embedding_dim)))  # Update input shape to (maxlen, embedding_dim)
model.add(layers.Dense(6, activation='softmax'))

# Step 5: Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 6: Train the model
batch_size = 32
epochs = 10

model.fit(
    np.array(X_train), np.array(y_train),
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)

# Step 7: Evaluate the model
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")





Epoch 1/10
414/414 [==============================] - 103s 231ms/step - loss: 1.0049 - accuracy: 0.7064 - val_loss: 1.0122 - val_accuracy: 0.6936
Epoch 2/10
414/414 [==============================] - 92s 221ms/step - loss: 0.9790 - accuracy: 0.7080 - val_loss: 1.0173 - val_accuracy: 0.6936
Epoch 3/10
414/414 [==============================] - 90s 216ms/step - loss: 0.9789 - accuracy: 0.7080 - val_loss: 1.0137 - val_accuracy: 0.6936
Epoch 4/10
414/414 [==============================] - 80s 193ms/step - loss: 0.9779 - accuracy: 0.7080 - val_loss: 1.0170 - val_accuracy: 0.6936
Epoch 5/10
414/414 [==============================] - 86s 208ms/step - loss: 0.9784 - accuracy: 0.7080 - val_loss: 1.0118 - val_accuracy: 0.6936
Epoch 6/10
414/414 [==============================] - 75s 182ms/step - loss: 0.9780 - accuracy: 0.7080 - val_loss: 1.0189 - val_accuracy: 0.6936
Epoch 7/10
414/414 [==============================] - 92s 221ms/step - loss: 0.9766 - accuracy: 0.7080 - val_loss: 1.0157 - val_a